In [1]:
import os
os.chdir("C:/Users/david/Desktop/Uni/potato-dry-matter-optics-ml")

from dotenv import load_dotenv

load_dotenv()  # lee el archivo .env

ROBOFLOW_API_KEY = os.environ["ROBOFLOW_API_KEY"]
os.environ["ROBOFLOW_API_KEY"] = ROBOFLOW_API_KEY

# Comprobación rápida
print("ROBOFLOW_API_KEY configurada?:", "ROBOFLOW_API_KEY" in os.environ)

ROBOFLOW_API_KEY configurada?: True


In [2]:
from inference_sdk import InferenceHTTPClient

CLIENT = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key=ROBOFLOW_API_KEY
)

In [3]:
IMAGE_PATH = "data/input/raw/raw_images/dani.jpg"

result = CLIENT.infer(IMAGE_PATH, model_id="coco-dataset-vdnr1/23")

In [4]:
print(result)

{'inference_id': '283ed6f0-669c-4669-bb4b-03b7bd63b6f6', 'time': 0.12278516998048872, 'image': {'width': 1200, 'height': 1600}, 'predictions': [{'x': 622.5, 'y': 949.5, 'width': 1127.0, 'height': 1299.0, 'confidence': 0.9612134695053101, 'class': 'person', 'points': [{'x': 557.5, 'y': 302.5}, {'x': 555.0, 'y': 305.0}, {'x': 552.5, 'y': 305.0}, {'x': 550.0, 'y': 307.5}, {'x': 547.5, 'y': 307.5}, {'x': 545.0, 'y': 310.0}, {'x': 540.0, 'y': 310.0}, {'x': 535.0, 'y': 315.0}, {'x': 532.5, 'y': 315.0}, {'x': 530.0, 'y': 317.5}, {'x': 527.5, 'y': 317.5}, {'x': 525.0, 'y': 320.0}, {'x': 522.5, 'y': 320.0}, {'x': 515.0, 'y': 327.5}, {'x': 512.5, 'y': 327.5}, {'x': 510.0, 'y': 330.0}, {'x': 507.5, 'y': 330.0}, {'x': 500.0, 'y': 337.5}, {'x': 497.5, 'y': 337.5}, {'x': 480.0, 'y': 355.0}, {'x': 480.0, 'y': 357.5}, {'x': 477.5, 'y': 360.0}, {'x': 477.5, 'y': 362.5}, {'x': 472.5, 'y': 367.5}, {'x': 472.5, 'y': 370.0}, {'x': 470.0, 'y': 372.5}, {'x': 470.0, 'y': 377.5}, {'x': 467.5, 'y': 380.0}, {'x'

In [5]:
from PIL import Image, ImageDraw
import numpy as np
import os
from typing import List, Tuple, Optional, Dict, Any

In [6]:
def _prediction_to_points(pred: Dict[str, Any]) -> Optional[List[Tuple[float, float]]]:
    """
    Extrae puntos de contorno si existen en el JSON de Roboflow.
    """
    pts = pred.get("points") or pred.get("polygon") or pred.get("vertices") or pred.get("segmentation")
    if not pts:
        return None

    if isinstance(pts, list) and len(pts) > 0:
        if isinstance(pts[0], dict):
            out = []
            for p in pts:
                x = p.get("x")
                y = p.get("y")
                if x is not None and y is not None:
                    out.append((float(x), float(y)))
            return out if len(out) >= 3 else None

        if isinstance(pts[0], (list, tuple)) and len(pts[0]) >= 2:
            out = [(float(p[0]), float(p[1])) for p in pts]
            return out if len(out) >= 3 else None

    return None


def polygon_to_mask(points: List[Tuple[float, float]], w: int, h: int) -> np.ndarray:
    """
    Rasteriza un polígono a máscara binaria.
    """
    mask_img = Image.new("L", (w, h), 0)
    draw = ImageDraw.Draw(mask_img)
    pts_int = [(int(x), int(y)) for x, y in points]
    draw.polygon(pts_int, outline=1, fill=1)
    return np.array(mask_img, dtype=np.uint8)


def _normalize_mask(m: Any, w: int, h: int) -> Optional[np.ndarray]:
    """
    Convierte un campo 'mask' del JSON a np.ndarray (h, w) binario.
    """
    if m is None:
        return None

    if isinstance(m, np.ndarray):
        arr = m
    elif isinstance(m, (list, tuple)):
        arr = np.array(m)
    elif isinstance(m, dict) and "data" in m:
        # por si viene como {data: [...]}
        arr = np.array(m["data"])
    else:
        try:
            arr = np.array(m)
        except:
            return None

    # Asegurar 2D
    if arr.ndim == 3:
        # posibles formatos raros
        if arr.shape[-1] == 1:
            arr = arr[..., 0]
        elif arr.shape[0] == 1:
            arr = arr[0]

    if arr.ndim != 2:
        return None

    arr = (arr > 0).astype(np.uint8)

    # Ajuste de tamaño si hiciera falta
    if arr.shape != (h, w):
        if arr.size == h * w:
            arr = arr.reshape((h, w))
        else:
            mask_img = Image.fromarray(arr * 255)
            mask_img = mask_img.resize((w, h), resample=Image.NEAREST)
            arr = (np.array(mask_img) > 0).astype(np.uint8)

    return arr


def extract_instance_masks_from_result(result: Dict[str, Any], w: int, h: int) -> List[np.ndarray]:
    """
    Intenta extraer máscaras por instancia desde result.
    Prioriza:
      1) points -> máscara
      2) mask directo
    """
    masks = []
    preds = result.get("predictions", []) if isinstance(result, dict) else []

    # 1) Si hay polígonos
    for pred in preds:
        points = _prediction_to_points(pred)
        if points:
            masks.append(polygon_to_mask(points, w, h))

    if masks:
        return masks

    # 2) Si hay máscaras directas
    for pred in preds:
        nm = _normalize_mask(pred.get("mask") or pred.get("segmentation_mask"), w, h)
        if nm is not None:
            masks.append(nm)

    return masks


def mask_boundary(mask: np.ndarray) -> np.ndarray:
    """
    Calcula borde de una máscara binaria sin OpenCV.
    """
    m = mask.astype(bool)
    h, w = m.shape

    up    = np.zeros_like(m); up[1:, :] = m[:-1, :]
    down  = np.zeros_like(m); down[:-1, :] = m[1:, :]
    left  = np.zeros_like(m); left[:, 1:] = m[:, :-1]
    right = np.zeros_like(m); right[:, :-1] = m[:, 1:]

    interior = up & down & left & right & m
    boundary = m & (~interior)
    return boundary.astype(np.uint8)


def largest_square_in_mask(mask: np.ndarray) -> Tuple[int, int, int]:
    """
    Mayor cuadrado de 1s en máscara (DP).
    Devuelve (x1, y1, size).
    """
    h, w = mask.shape
    dp = np.zeros((h, w), dtype=np.int32)
    best_size = 0
    best_br = (0, 0)

    m = mask.astype(bool)

    for i in range(h):
        for j in range(w):
            if m[i, j]:
                if i == 0 or j == 0:
                    dp[i, j] = 1
                else:
                    dp[i, j] = 1 + min(dp[i-1, j], dp[i, j-1], dp[i-1, j-1])

                if dp[i, j] > best_size:
                    best_size = int(dp[i, j])
                    best_br = (i, j)

    if best_size == 0:
        return 0, 0, 0

    br_y, br_x = best_br
    x1 = br_x - best_size + 1
    y1 = br_y - best_size + 1
    return x1, y1, best_size


In [7]:
from typing import Dict, Any, Optional, Tuple
from PIL import Image, ImageDraw
import numpy as np
import os


def save_contour_plus_inner_square(
    image_path: str,
    result: Dict[str, Any],
    output_path: str,
    margin: int = 0,
) -> Optional[Tuple[int, int, int, int]]:
    """
    Desa una imatge amb:
      - el contorn combinat de totes les màscares (en vermell),
      - el quadrat màxim trobat dins la millor màscara (en verd),
      - i, si margin > 0, un quadrat intern amb el marge restat (en blau).

    Retorna
    -------
    best_xyxy : tuple o None
        Coordenades (x1, y1, x2, y2) del quadrat màxim sense marge.
    """
    image = Image.open(image_path).convert("RGB")
    w, h = image.size

    masks = extract_instance_masks_from_result(result, w, h)

    # Si no hi ha màscares, desem la imatge tal qual i retornem None
    if not masks:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        image.save(output_path)
        return None

    # Borde combinat + millor quadrat global
    global_boundary = np.zeros((h, w), dtype=np.uint8)
    best_size = 0
    best_xyxy: Optional[Tuple[int, int, int, int]] = None

    for mask in masks:
        boundary = mask_boundary(mask)
        global_boundary = np.maximum(global_boundary, boundary)

        x1, y1, size = largest_square_in_mask(mask)
        if size > best_size:
            best_size = size
            best_xyxy = (x1, y1, x1 + size, y1 + size)

    # Dibuixem a la imatge (sense matplotlib / cv2)
    vis = image.copy()
    draw = ImageDraw.Draw(vis)

    # Contorn vermell píxel a píxel
    if global_boundary.any():
        red = (255, 0, 0)
        ys, xs = np.where(global_boundary == 1)
        for x, y in zip(xs.tolist(), ys.tolist()):
            vis.putpixel((x, y), red)

    inner_xyxy: Optional[Tuple[int, int, int, int]] = None

    # Quadrat màxim verd
    if best_xyxy is not None:
        x1, y1, x2, y2 = best_xyxy
        draw.rectangle([x1, y1, x2, y2], outline="lime", width=3)

        # Quadrat intern amb marge restat (si margin > 0)
        if margin > 0:
            ix1 = x1 + margin
            iy1 = y1 + margin
            ix2 = x2 - margin
            iy2 = y2 - margin

            # Només si continua sent un quadrat/rectangle vàlid
            if ix2 > ix1 and iy2 > iy1:
                inner_xyxy = (ix1, iy1, ix2, iy2)
                # El pintem en blau perquè es vegi diferent
                draw.rectangle(inner_xyxy, outline="blue", width=3)

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    vis.save(output_path)

    # Per compatibilitat, retornem només el quadrat màxim
    return best_xyxy


def save_inner_square_crop(
    image_path: str,
    square_xyxy: Optional[Tuple[int, int, int, int]],
    margin: int,
    output_path: str,
) -> Optional[Tuple[int, int, int, int]]:
    """
    Desa una nova imatge que és només l'àrea interna del quadrat màxim,
    aplicant-hi un marge cap endins.

    Paràmetres
    ----------
    image_path : str
        Ruta a la imatge original.
    square_xyxy : tuple o None
        Coordenades (x1, y1, x2, y2) del quadrat màxim (sense marge).
    margin : int
        Nombre de píxels a restar per cada costat (cap endins).
    output_path : str
        Ruta on desar la imatge retallada.

    Retorna
    -------
    inner_xyxy : tuple o None
        Coordenades del quadrat intern usat per al retall, o None si no és vàlid.
    """
    if square_xyxy is None:
        # No hi ha quadrat màxim, no fem res
        return None

    image = Image.open(image_path).convert("RGB")
    w, h = image.size

    x1, y1, x2, y2 = square_xyxy

    # Apliquem marge cap endins
    ix1 = x1 + margin
    iy1 = y1 + margin
    ix2 = x2 - margin
    iy2 = y2 - margin

    # Ens assegurem que el resultat és vàlid
    if ix2 <= ix1 or iy2 <= iy1:
        return None

    # També ens assegurem que no sortim dels límits
    ix1 = max(0, ix1)
    iy1 = max(0, iy1)
    ix2 = min(w, ix2)
    iy2 = min(h, iy2)

    inner_xyxy = (ix1, iy1, ix2, iy2)

    cropped = image.crop(inner_xyxy)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cropped.save(output_path)

    return inner_xyxy


In [8]:
OUTPUT_PATH = "data/input/processed_images/dani_visualize.jpg"

best_square_xyxy = save_contour_plus_inner_square(
    IMAGE_PATH,
    result,
    OUTPUT_PATH,
    margin=10,  # o el marge que vulguis
)


print("=== COORDENADAS DEL CUADRADO INTERNO (x1, y1, x2, y2) ===")
print(best_square_xyxy)

print("=== IMAGEN GUARDADA ===")
print(OUTPUT_PATH)


=== COORDENADAS DEL CUADRADO INTERNO (x1, y1, x2, y2) ===
(237, 888, 946, 1597)
=== IMAGEN GUARDADA ===
data/input/processed_images/dani_visualize.jpg


In [ ]:
CROPPED_OUTPUT_PATH = "data/input/processed_images/dani_cut.jpg"
margin = 10  # mateix marge que hagis passat abans

inner_xyxy = save_inner_square_crop(
    IMAGE_PATH,
    best_square_xyxy,
    margin,
    CROPPED_OUTPUT_PATH,
)

print("Quadrat intern:", inner_xyxy)
print("Imatge interna desada a:", CROPPED_OUTPUT_PATH)


Quadrat intern: (247, 898, 936, 1587)
Imatge interna desada a: data/input/processed_images/dani_cut.jpg
